Creation d'une DB PERE-NOEL avec postgres

Creation des tables : T_lettres / T_enfants / T_pays / T_contenus
et d'une table de travail T_complete

In [2]:
import psycopg2
from zipfile import ZipFile
import os

In [3]:
def chemin_fichier()-> str :
    """
        Donne le nom du chemin complet du fichier actuel (.py) 
    """
    chemin=os.getcwd() #methode pour tourner avec jupyter
    #chemin= path.dirname(path.abspath(__file__)) #methode d'Alice avec <os>
    #chemin = str(Path(__file__).parent) #ma methode avec <path>
    print(chemin)
    return chemin

chemin_f=chemin_fichier()

/home/tatindom/Santa-Data/santa-Groupe-DZJC/Dossier_de_travail


In [4]:
# etablir une connection
def connexion_DB(fichierDB):
    try:
        conn = psycopg2.connect(user = "santa",
                                password = "claus",
                                host = "localhost",
                                port = "5432:5432",
                                database = fichierDB
                                )

        cur = conn.cursor()
        # Print PostgreSQL Connection properties
        print ( "Proprietes : ",conn.get_dsn_parameters(),"\n")

        # Print PostgreSQL version
        cur.execute("SELECT version();")
        record = cur.fetchone()
        print("Version de PostgreSQL : ", record,"\n")
        cur.close()
    except (Exception, psycopg2.Error) as error :
        print ("Error while connecting to PostgreSQL", error)
    
    return conn
                

In [5]:
def deconnexion_DB(conn):
#closing database connection.
    if(conn):
        conn.close()
        print("PostgreSQL connection is closed")

In [ ]:
def create_table(conn, create_table_psg : str,table_name):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(create_table_psg)
        print(f"Table created successfully in PostgreSQL : {table_name}")
    except (Exception, psycopg2.DatabaseError) as error :
        print(f"Error while creating PostgreSQL table : {table_name}", error)
    finally:
        #fermer le curseur
        if(conn):
            c.close()

In [ ]:
# MAIN
# CONNECTION A LA BASE

conn=connexion_DB("santa_data")
#print(conn)

In [ ]:
#creation de tables
if conn is not None:
    #T_lettres
    nm_table='T_lettres'
    
    c = conn.cursor()
    
    c.execute(f"DELETE FROM {nm_table}")
    
    c.execute(f"DROP TABLE IF EXISTS {nm_table}")
    
    create_table_query = f'''
            CREATE TABLE IF NOT EXISTS {nm_table}
            (ID_l SERIAL PRIMARY KEY NOT NULL,
            Nom_lettre TEXT NOT NULL
            ); 
            '''
    create_table(conn,create_table_query,nm_table)
    conn.commit()


In [ ]:
if conn is not None:
    
    #T_enfants
    nm_table='T_enfants'
    
    c = conn.cursor()
    
    #suppression des enregistrements de la T_complete avant de refaire tourne le programme
    #c.execute(f"DELETE FROM {nm_table}")
    
    c.execute(f"DROP TABLE IF EXISTS {nm_table}")
    
    create_table_query = f'''
            CREATE TABLE IF NOT EXISTS {nm_table}
            (ID_e SERIAL PRIMARY KEY,
            Nom_enfant TEXT NOT NULL,
            Age FLOAT,
            ID_p INT,
            ID_l INT
            ); 
            
            '''
    create_table(conn,create_table_query,nm_table)
    conn.commit()

In [ ]:
#creation de tables
if conn is not None:
    
    #T_contenu_lettres
    nm_table='T_contenus'
    
    c = conn.cursor()
    
    c.execute(f"DELETE FROM {nm_table}")
    
    c.execute(f"DROP TABLE IF EXISTS {nm_table}")
    
    create_table_query = f'''
            CREATE TABLE IF NOT EXISTS {nm_table}
            (ID_c SERIAL PRIMARY KEY,
            ID_l INT,
            Contenu TEXT,
            Numero TEXT
            ); 
            '''
    create_table(conn,create_table_query,nm_table)
    conn.commit()
    

In [ ]:
if conn is not None:
        
    #T_pays
    nm_table='T_pays'
    
    c = conn.cursor()
    
    c.execute(f"DELETE FROM {nm_table}")
    
    c.execute(f"DROP TABLE IF EXISTS {nm_table}")
    
    create_table_query = f'''
            CREATE TABLE IF NOT EXISTS {nm_table}
            (ID_p SERIAL PRIMARY KEY,
            Pays TEXT
            ); 
            '''
    create_table(conn,create_table_query,nm_table)
    conn.commit()
    

In [ ]:
if conn is not None:
    
    #table de travail T_complete
    nm_table='T_complete'
    
    c = conn.cursor()
    
    c.execute(f"DELETE FROM {nm_table}")
    
    c.execute(f"DROP TABLE IF EXISTS {nm_table}")
    
    create_table_query = f'''
            CREATE TABLE IF NOT EXISTS {nm_table}
            (ID SERIAL PRIMARY KEY,
            id_l INT,
            nom_lettre TEXT,
            nom_enfant TEXT,
            age TEXT,
            Pays TEXT,
            Contenu TEXT,
            Numero TEXT
            ); 
            '''
    create_table(conn,create_table_query,nm_table)
    conn.commit()

In [ ]:
deconnexion_DB(conn)

In [ ]:
#insertion des données Nom_des_fichiers_lettres dans la T_lettres

conn=connexion_DB("santa_data")
try:
    cursor = conn.cursor()

    path_dossier_letters=chemin_fichier() +'/Partie_1/lettres_pere_noel'
    #print(path_dossier_letters)
#'/home/tatindom/Santa-Data/santa-Groupe-DZJC/Partie_1/lettres_pere_noel')
    
    #recuperation du nom des fichiers <lettre---.pnoel>
    list_letters=os.listdir(path_dossier_letters)
    list_letters.sort()
       
    count=0
    
    for f_name in list_letters :
        insert_data_query = f""" 
                        INSERT INTO T_lettres
                        (ID_l,Nom_lettre) 
                        VALUES (default,'{f_name}')
                        """
        cursor.execute(insert_data_query)
        conn.commit()
        count += cursor.rowcount
        
    print (count, "Record inserted successfully into mobile table")
       
except (Exception, psycopg2.Error) as error :
    if(conn):
        print("Failed to insert record into table T_lettres", error)

finally:
    #closing database connection.
    if(conn):
        cursor.close()
        deconnexion_DB(conn)

In [6]:
# controle des resultats
conn=connexion_DB("santa_data")
cursor = conn.cursor()

cursor.execute("SELECT * FROM T_lettres")
res=cursor.fetchall()
print("CONTROLE DE L'INSERTION\n")


print('\nID_l'.ljust(8),'nom_lettre'.ljust(25))
trait="-"*20
print(trait)
for row in res:
    print(str(row[0]).ljust(8),end='')
    print(str(row[1]).ljust(25))
    
cursor.close()
deconnexion_DB(conn)

Proprietes :  {'user': 'santa', 'dbname': 'santa_data', 'host': 'localhost', 'port': '5432:5432', 'tty': '', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 

Version de PostgreSQL :  ('PostgreSQL 12.1 (Debian 12.1-1.pgdg100+1) on x86_64-pc-linux-gnu, compiled by gcc (Debian 8.3.0-6) 8.3.0, 64-bit',) 

CONTROLE DE L'INSERTION


ID_l    nom_lettre               
--------------------
1       lettre_1.pnoel           
2       lettre_10.pnoel          
3       lettre_100.pnoel         
4       lettre_1000.pnoel        
5       lettre_1001.pnoel        
6       lettre_1002.pnoel        
7       lettre_1003.pnoel        
8       lettre_1004.pnoel        
9       lettre_1005.pnoel        
10      lettre_1006.pnoel        
11      lettre_1007.pnoel        
12      lettre_1008.pnoel        
13      lettre_1009.pnoel        
14      lettre_101.pnoel         
15      lettre_1010.pnoel        
16      lettre_1011.pnoel        
17 

1077    lettre_491.pnoel         
1078    lettre_492.pnoel         
1079    lettre_493.pnoel         
1080    lettre_494.pnoel         
1081    lettre_495.pnoel         
1082    lettre_496.pnoel         
1083    lettre_497.pnoel         
1084    lettre_498.pnoel         
1085    lettre_499.pnoel         
1086    lettre_5.pnoel           
1087    lettre_50.pnoel          
1088    lettre_500.pnoel         
1089    lettre_501.pnoel         
1090    lettre_502.pnoel         
1091    lettre_503.pnoel         
1092    lettre_504.pnoel         
1093    lettre_505.pnoel         
1094    lettre_506.pnoel         
1095    lettre_507.pnoel         
1096    lettre_508.pnoel         
1097    lettre_509.pnoel         
1098    lettre_51.pnoel          
1099    lettre_510.pnoel         
1100    lettre_511.pnoel         
1101    lettre_512.pnoel         
1102    lettre_513.pnoel         
1103    lettre_514.pnoel         
1104    lettre_515.pnoel         
1105    lettre_516.pnoel         
1106    lettre

     
#pour dezipper un dossier  
    # spécifiant le nom du fichier zip
file = "lettres_pere_noel.zip"

#ouvrir le fichier zip en mode lecture
with ZipFile(file, 'r') as zip: 
    # afficher tout le contenu du fichier zip
zip.printdir() 
    
#extraire tous les fichiers
print('extraction...') 
zip.extractall() 
print('Terminé!')
        

#recuperer la liste des fichiers d'un dossier
list_lettres=os.listdir('/home/tatindom/Santa-Data/santa-Groupe-DZJC/Partie_1/lettres_pere_noel')
print(list_lettres)